In [1]:
import uuid
from functools import partial

import evaluate
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments,
)


def is_subword(text, tokenized, tokenizer, index):
    word = tokenizer.convert_ids_to_tokens(tokenized["input_ids"][index])
    start_ind, end_ind = tokenized["offset_mapping"][index]
    word_ref = text[start_ind:end_ind]
    is_subword = len(word) != len(word_ref)
    return is_subword


def tokenize(example, labels2int, tokenizer, iob=True, ignore_subwords=True):

    text, labels = example["source_text"], example["privacy_mask"]

    i = 0
    token_labels = []

    tokenized = tokenizer(text, return_offsets_mapping=True, return_special_tokens_mask=True)
    start_token_to_label = {
        tokenized.char_to_token(label["start"]): (label["start"], label["end"], label["label"]) for label in labels
    }
    while i < len(tokenized["input_ids"]):
        if tokenized["special_tokens_mask"][i] == 1:
            token_labels.append(-100)
            i += 1
        elif i not in start_token_to_label:
            if ignore_subwords and is_subword(text, tokenized, tokenizer, i):
                token_labels.append(-100)
            else:
                token_labels.append(labels2int["O"])
            i += 1
        else:
            start, end, label = start_token_to_label[i]
            start_token = tokenized.char_to_token(start)
            assert start_token == i
            j = start_token
            while j < (len(tokenized["input_ids"]) - 1) and tokenized.token_to_chars(j).start < end:
                if j == start_token:
                    if iob:
                        token_labels.append(labels2int["B-" + label])
                    else:
                        token_labels.append(labels2int[label])
                elif ignore_subwords and is_subword(text, tokenized, tokenizer, j):
                    token_labels.append(-100)
                else:
                    if iob:
                        token_labels.append(labels2int["I-" + label])
                    else:
                        token_labels.append(labels2int[label])

                j += 1
            i = j
    tokenized["labels"] = token_labels
    return tokenized


def compute_metrics(eval_pred, label_list, seqeval_metric):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval_metric.compute(predictions=true_predictions, references=true_labels)
    results_flat = {f"{k}_f1": v["f1"] for k, v in results.items() if isinstance(v, dict)}
    results_flat.update(
        {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }
    )
    return results_flat


/usr/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-21 18:35:37.037172: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-21 18:35:37.052232: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-21 18:35:37.056890: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-21 18:35:37.068923: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use av

In [4]:
labels = [
    "BOD",
    "BUILDING",
    "CARDISSUER",
    "CITY",
    "COUNTRY",
    "DATE",
    "DRIVERLICENSE",
    "EMAIL",
    "GEOCOORD",
    "GIVENNAME1",
    "GIVENNAME2",
    "IDCARD",
    "IP",
    "LASTNAME1",
    "LASTNAME2",
    "LASTNAME3",
    "PASS",
    "PASSPORT",
    "POSTCODE",
    "SECADDRESS",
    "SEX",
    "SOCIALNUMBER",
    "STATE",
    "STREET",
    "TEL",
    "TIME",
    "TITLE",
    "USERNAME",
]

labels = [f"I-{label}" for label in labels] + [f"B-{label}" for label in labels] + ["O"]
label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}
pretrained_name = "google/mobilebert-uncased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_name)
model = AutoModelForTokenClassification.from_pretrained(pretrained_name, num_labels=len(labels), id2label=id2label)

ds = load_dataset("ai4privacy/pii-masking-300k")
ds = ds.filter(lambda x: x["language"] == "English", num_proc=4)
ds = ds.map(
    partial(tokenize, labels2int=label2id, tokenizer=tokenizer, iob=True, ignore_subwords=True),
    batched=False,
    remove_columns=[
        "source_text",
        "target_text",
        "privacy_mask",
        "span_labels",
        "mbert_text_tokens",
        "mbert_bio_labels",
        "id",
        "language",
        "set",
    ],
    num_proc=8,
).remove_columns(["offset_mapping"])

training_arguments = TrainingArguments(
    output_dir="output_MOBILE_BERT",
    max_steps=40000,
    eval_steps=5000,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    overwrite_output_dir=True,
    warmup_ratio=0.2,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy="steps",
    eval_strategy="steps",
    load_best_model_at_end=True,
    save_total_limit=1,
    save_steps=10000,
    lr_scheduler_type="cosine",
    warmup_steps=3000,
    metric_for_best_model="f1",
    greater_is_better=True,
    torch_compile=False,

)
trainer = Trainer(
    model,
    training_arguments,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    data_collator=DataCollatorForTokenClassification(tokenizer),
    tokenizer=tokenizer,
    compute_metrics=partial(compute_metrics, label_list=labels, seqeval_metric=evaluate.load("seqeval")),
)


Some weights of MobileBertForTokenClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
max_steps is given, it will override any value given in num_train_epochs


In [5]:
trainer.train()


  1%|▏         | 500/40000 [02:01<2:29:06,  4.42it/s]

{'loss': 3.9588, 'grad_norm': 1.8109838962554932, 'learning_rate': 3.3333333333333333e-06, 'epoch': 0.53}


  3%|▎         | 1001/40000 [04:02<2:31:08,  4.30it/s]

{'loss': 0.2998, 'grad_norm': 1.2737737894058228, 'learning_rate': 6.666666666666667e-06, 'epoch': 1.07}


  4%|▍         | 1500/40000 [06:03<2:43:32,  3.92it/s]

...

                                                       
100%|██████████| 40000/40000 [2:45:01<00:00,  4.21it/s]

{'eval_loss': 0.06687254458665848, 'eval_BOD_f1': 0.9671848013816926, 'eval_BUILDING_f1': 0.9815005138746146, 'eval_CARDISSUER_f1': 0.0, 'eval_CITY_f1': 0.9705593719332678, 'eval_COUNTRY_f1': 0.9685138539042821, 'eval_DATE_f1': 0.9226327944572749, 'eval_DRIVERLICENSE_f1': 0.9569736021617129, 'eval_EMAIL_f1': 0.9851541682527598, 'eval_GEOCOORD_f1': 0.9677419354838711, 'eval_GIVENNAME1_f1': 0.8613455510007235, 'eval_GIVENNAME2_f1': 0.7895716945996275, 'eval_IDCARD_f1': 0.939365496527126, 'eval_IP_f1': 0.9852330410706046, 'eval_LASTNAME1_f1': 0.8338430173292558, 'eval_LASTNAME2_f1': 0.7272727272727272, 'eval_LASTNAME3_f1': 0.6918918918918918, 'eval_PASS_f1': 0.933046683046683, 'eval_PASSPORT_f1': 0.9550030819806863, 'eval_POSTCODE_f1': 0.9765013054830287, 'eval_SECADDRESS_f1': 0.9680789320951828, 'eval_SEX_f1': 0.9722222222222222, 'eval_SOCIALNUMBER_f1': 0.9524744697564809, 'eval_STATE_f1': 0.9787023278850916, 'eval_STREET_f1': 0.9688796680497925, 'eval_TEL_f1': 0.970946113732307, 'eval_T

100%|██████████| 40000/40000 [2:45:01<00:00,  4.04it/s]

{'train_runtime': 9901.8917, 'train_samples_per_second': 129.268, 'train_steps_per_second': 4.04, 'train_loss': 0.0695130168452859, 'epoch': 42.78}


TrainOutput(global_step=40000, training_loss=0.0695130168452859, metrics={'train_runtime': 9901.8917, 'train_samples_per_second': 129.268, 'train_steps_per_second': 4.04, 'total_flos': 3.328130452343112e+16, 'train_loss': 0.0695130168452859, 'epoch': 42.780748663101605})

{'eval_loss': 0.06687254458665848, 'eval_BOD_f1': 0.9671848013816926, 'eval_BUILDING_f1': 0.9815005138746146, 'eval_CARDISSUER_f1': 0.0, 'eval_CITY_f1': 0.9705593719332678, 'eval_COUNTRY_f1': 0.9685138539042821, 'eval_DATE_f1': 0.9226327944572749, 'eval_DRIVERLICENSE_f1': 0.9569736021617129, 'eval_EMAIL_f1': 0.9851541682527598, 'eval_GEOCOORD_f1': 0.9677419354838711, 'eval_GIVENNAME1_f1': 0.8613455510007235, 'eval_GIVENNAME2_f1': 0.7895716945996275, 'eval_IDCARD_f1': 0.939365496527126, 'eval_IP_f1': 0.9852330410706046, 'eval_LASTNAME1_f1': 0.8338430173292558, 'eval_LASTNAME2_f1': 0.7272727272727272, 'eval_LASTNAME3_f1': 0.6918918918918918, 'eval_PASS_f1': 0.933046683046683, 'eval_PASSPORT_f1': 0.9550030819806863, 'eval_POSTCODE_f1': 0.9765013054830287, 'eval_SECADDRESS_f1': 0.9680789320951828, 'eval_SEX_f1': 0.9722222222222222, 'eval_SOCIALNUMBER_f1': 0.9524744697564809, 'eval_STATE_f1': 0.9787023278850916, 'eval_STREET_f1': 0.9688796680497925, 'eval_TEL_f1': 0.970946113732307, 'eval_TIME_f1': 0.9695451549110086, 'eval_TITLE_f1': 0.9568607068607069, 'eval_USERNAME_f1': 0.9333333333333333, 'eval_precision': 0.9410922874603543, 'eval_recall': 0.952028798372266, 'eval_f1': 0.9465289529478127, 'eval_accuracy': 0.992088277633207, 'eval_runtime': 28.7216, 'eval_samples_per_second': 276.656, 'eval_steps_per_second': 8.669, 'epoch': 42.78}
